# DREAM Challenge - seq2exp by BMDS Lab, QUT

## Data Preparation

### Install dependencies, and separately, dna2vec

In [ ]:
!pip install torch pandas numpy
!git clone https://github.com/pnpnpn/dna2vec
!pip install -r dna2vec/requirements.txt
!pip install --upgrade numpy
!pip install -e ./dna2vec
!pip install scikit-learn==1.0.2

In [ ]:
!pip uninstall gensim
!pip install git+https://github.com/RaRe-Technologies/gensim.git@78da89a29acc04af76a807a3693c08f09acf8ed8

In [ ]:
# Modify line 55 of /mnt/ssd1/seq2exp/gensim/gensim/models/ldamodel.py
#  From: from scipy.misc import logsumexp
#  To: from scipy.special import logsumexp

!sed -i '55s/.*/    from scipy.special import logsumexp/' lib/python3.8/site-packages/gensim/models/ldamodel.py

In [ ]:
from dna2vec.multi_k_model import MultiKModel

In [ ]:
!mkdir pytorch-tensors-all
!mkdir pytorch-tensors-test-set

### Import dependencies

In [ ]:
import csv
import sys
import pickle
import math

from time import time

import torch
import pandas as pd
import numpy as np

from dna2vec.multi_k_model import MultiKModel

### Prepare trimmed training sequences

In [ ]:
max_seq = 10000000
i = 0

with open('train_sequences.txt','r') as inFile, open('train_sequences.fasta','w') as outFile:
    start = time()
    
    for line in inFile:
        data = line.rstrip().split("\t") 
        ID = "> "+data[0]+","+data[1]
        trimmed_seq = data[0][17:-13]
        if "N" in trimmed_seq:
            continue

        i+=1
        
        outFile.write(ID+"\n"+trimmed_seq+"\n")
        
        if i>=max_seq:
            break

    end = time()
    print(end-start)

### Preparing dna2vec encodings

In [ ]:
# download dna2vec model
filepath = 'dna2vec-20161219-0153-k3to8-100d-10c-29320Mbp-sliding-Xat.w2v'
if not os.path.exists(filepath):
    import urllib.request
    urllib.request.urlretrieve(
        "https://github.com/pnpnpn/dna2vec/blob/master/pretrained/dna2vec-20161219-0153-k3to8-100d-10c-29320Mbp-sliding-Xat.w2v", 
        "dna2vec-20161219-0153-k3to8-100d-10c-29320Mbp-sliding-Xat.w2v"
    )
mk_model = MultiKModel(filepath)

In [ ]:
# test dna2vec
' '.join(map(str, mk_model.vector('ATCG')))

### Load sequences

In [ ]:
# the file is formatted as:
'''
> TGCATTTTTTTCACATCTCTTTGCCACGGGGTGAAGGATAGGATGGTATCCCCCCAGGCGAAGGACATCTGTGGGGATGGTTAGGTCAGGTGATATCGGTTACGGCTGTT,11
TCTTTGCCACGGGGTGAAGGATAGGATGGTATCCCCCCAGGCGAAGGACATCTGTGGGGATGGTTAGGTCAGGTGATATC
> TGCATTTTTTTCACATCTATGTTGCGTTAGAACGATATTGGAACACTTGTCAACAAGCTCATCTGAACTAATAGAGATGTATTCATAGGCTTCAGGTGGTTACGGCTGTT,6
TATGTTGCGTTAGAACGATATTGGAACACTTGTCAACAAGCTCATCTGAACTAATAGAGATGTATTCATAGGCTTCAGGT
...

'''


with open('train_sequences.fasta', 'r') as fp:
    data = {'sequence' : [], 'score' : []}
    for line in fp:
        fields = line.strip()[2:].split(',')
        data['sequence'].append(fields[0])
        data['score'].append(fields[1])
        next(fp) # skip every even line
    
    df = pd.DataFrame(data)
    df['sequence_trimmed'] = df['sequence'].str[17:-13]

### Function for computing embedding

In [ ]:
def overlapping_kmer_encoding_dna2vec(seq, k = 8):
    # note: k is k until there is only k = [3,k] remaining positions in the sequence.

    for i in range(0, len(seq) - k + 1):
        sseq = seq[i:i+k]
        if len(sseq) < 3 or len(sseq) > 8:
            #print(f'error with `{sseq}`. length needs to be 3-8 (inc.)')
            continue
        #print( mk_model.vector(sseq))
        yield mk_model.vector(sseq)

### Generate training Tensors

In [ ]:
# PyTorch tensors

set_size = 1000

# value for n
n = 300_000
n = 300_000_000

# values for k
k = 3

df_n = df.iloc[0:n, :]
longest_sequence_trimmed = df_n['sequence_trimmed'].apply(len).max()
print(f'longest_sequence_trimmed: {longest_sequence_trimmed}')

X = []
y = []

file_idx = 0

for rowIdx, row in df_n.iterrows():
    data = []

    for embedIdx, embed in enumerate(overlapping_kmer_encoding_dna2vec(row['sequence_trimmed'], k=k)):
        data.append(list(embed))

    while len(data) < longest_sequence_trimmed - k + 1:
        data.append(np.zeros(100))

    X.append(data)
    y.append(float(row['score']))
    
    if len(X) > 0 and len(X) % set_size == 0:

        _X = torch.tensor(X, dtype=torch.float32)
        _y = torch.tensor(y, dtype=torch.float32)

        print(file_idx, _X.shape, _y.shape)

        torch.save(_X, f'jake/pytorch-tensors-all/Batch{file_idx:0>5}X.pt')
        torch.save(_y, f'jake/pytorch-tensors-all/Batch{file_idx:0>5}y.pt')
        
        X = []
        y = []
        file_idx += 1

### Generate testing Tensors

In [ ]:
# PyTorch tensors, for test

seqs = []
with open('test_sequences.txt', 'r') as fp:
    data = {'sequence' : [], 'score' : []}
    for line in fp:
        seqs.append(line.split('\t')[0].strip()[17:-13])
print(seqs[0:3])
set_size = 1000
k = 3

longest_sequence_trimmed = max([
    len(x)
    for x in seqs
])
print(f'longest_sequence_trimmed: {longest_sequence_trimmed}')

_train = torch.load('pytorch-tensors-all/Batch00001X.pt')
longest_sequence_trimmed = _train.shape[1] + k - 1
print(f'training tensor shape: {_train.shape}')
print(f'forced longest_sequence_trimmed to be: {longest_sequence_trimmed - k + 1}, to match the training set')


X = []

file_idx = 0

for rowIdx, seq in enumerate(seqs):
    data = []

    for embedIdx, embed in enumerate(overlapping_kmer_encoding_dna2vec(seq, k=k)):
        data.append(list(embed))

    while len(data) < longest_sequence_trimmed - k + 1:
        data.append(np.zeros(100))

    X.append(data)
    
    if len(X) > 0 and len(X) % set_size == 0 or rowIdx == (len(seqs) - 1):

        _X = torch.tensor(X, dtype=torch.float32)

        print(file_idx, _X.shape)

        torch.save(_X, f'pytorch-tensors-test-set/Batch{file_idx:0>5}X.pt')
        
        X = []
        file_idx += 1